#**Sensor Push Access Notebook**#
# David B. Henthorn, Dept. of Chemical Engineering,
Rose-Hulman Institute of Technology

<img style="float: right;" src="https://raw.githubusercontent.com/henthornlab/ProcessAnalytics/master/RHITlogo.png">




In [78]:
import requests
import ast
from io import StringIO
import pandas as pd
from datetime import timedelta  
import plotly.express as px

Accessing data requires a token. But before we get the token, we need to first supply a username and password to be authorized. 

In [79]:
authHeaders = {'accept': 'application/json', 'Content-Type': 'application/json'}
authUrl = "https://api.sensorpush.com/api/v1/oauth/authorize"
authBody = '{"email": "xxxxx@xxx", "password": "yyyyyyyyy"}'

authResponse = requests.post(authUrl, headers=authHeaders, data = authBody)

if authResponse.status_code != 200:
    print('# Request failed with code:', authResponse.status_code, authResponse.text)
else:
    print('Authorized')

Authorized


After authorization, we now ask for a token.

In [80]:
tokenHeaders = {'accept': 'application/json', 'Content-Type': 'application/json'}
tokenUrl = "https://api.sensorpush.com/api/v1/oauth/accesstoken"
tokenBody = authResponse.text


tokenResponse = requests.post(tokenUrl, headers=tokenHeaders, data = tokenBody)

if tokenResponse.status_code != 200:
    print('# Request failed with code:', tokenResponse.status_code, tokenResponse.text)
else:
    token = ast.literal_eval(tokenResponse.text)['accesstoken']
    print("Success! Token is", token)

Success! Token is eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXlJZCI6IkFTSUFSVk5OWEhYN1dKTTVQQkhXIiwic2VjcmV0QWNjZXNzS2V5IjoiQkpJN2M5QjcrUWxYUlNFaGU0Slc5RmFaK0c5QTZ4NmZWTHJOQitjYiIsInNlc3Npb25Ub2tlbiI6IkZ3b0daWEl2WVhkekVOSC8vLy8vLy8vLy93RWFETDhlclJXa1hMU3NtcFp2WHlLNUE3bTFCNG5HNVlkUW1CUnF6R205V1hzckxxRVR3T3BRT1YzaUcvT0dyVU5PT3BOUjdESW5CaThZeGlrdW9mN2o0WEU4SmQzWVVEd0RDWEFyUTJnaUltYXhTZExQNWx4bGhDWTcycXJaNHFxTE1KZDZLdE1IaVNmeUxzSWgwWHNwSmxBUTZWTlRHTHVhK1BjVlp5ZTZJNmRFc3FDcFRleTZwTDZkNHNIOERwTkNVanhlNG44SE9wNWcrdDBoVkNJdkVXeWoxWkU3MmNUTkFqT3JWTEQ3K3BtazcrK0l2eWd3ZksyUW10SnR6aHpiWm5FVzliUjZ0c3ZDTXJhOWVCREZJVXFhSUtBNDViVjBSdGJuTGtYYUdPQVlpZjdJeU9waWdpWkt4WEd1TDhJVkx5TUVJWmVmd293dzBxNkx1WkRrYTJvVDFtbW42WnJTVmNaYVM4K09hMW1VN3VGcC9yOGtrR0l1ek00cTYrM09COWNYTDM4bXVjaWIrdnF6ViszQ2hwS3JucmFteW1XR1Azdm53UFBtZFFvS0pZU1h6cUNoTE0ySkYvRG90aXU1T3JYd2paa2R0S2wwSFk4SzJXWHcvVm9KaEk4V3FGNHRhdnVtVk45QUlNdWRMQ0hWakVFQWZXdzNzZXpXallMS3Fua20yK0U5WCtpWTM3K05pZ1JWMit3aGtBSzVldCtkeHk1b2JwWkJHYWJOZUJIZ2wxUUo3K

We can ask for the last x number of data points using the /samples endpoint. The provided data is easier work with in CSV form, ask for that with format:csv 

In [81]:
queryHeaders = {'accept': 'application/json', 'Authorization': token}
queryUrl = "https://api.sensorpush.com/api/v1/samples"
queryBody = '{ "limit": 1000, "format":"csv" }'

#print(queryHeaders)

queryResponse = requests.post(queryUrl, headers=queryHeaders, data = queryBody)

if queryResponse.status_code != 200:
    print('# Request failed with code:', queryResponse.status_code, queryResponse.text)
else:
    print('Success!')

Success!


Let's take the string of data and format it into a Pandas dataframe. First pass it though StringIO to clean it up.  

Next, adjust the provided time (UTC) to match the local timezone.

In [82]:
    df = pd.read_csv(StringIO(queryResponse.text), skiprows=2)
    df['Time'] = pd.to_datetime(df['observed'], infer_datetime_format=True) + timedelta(hours = -5)

Time to plot temperature and then humidity.

In [83]:
fig = px.scatter(df, x = 'Time', y = 'temperature')
fig.update_layout(
    title="UOLAB Temperatures",
    xaxis_title='Time (Eastern)',
    yaxis_title="Temperature",
    font=dict(
        family="Arial",
        size=16
    )
)
fig.show()

In [84]:
fig2 = px.scatter(df, x = 'Time', y = 'humidity')
fig2.update_layout(
    title="UOLAB Humidity",
    xaxis_title='Time (Eastern)',
    yaxis_title="Humidity%",
    font=dict(
        family="Arial",
        size=16
    )
)
fig2.show()